In [8]:
try:
    from chgnet.model import CHGNet
except ImportError:
    # install CHGNet (only needed on Google Colab or if you didn't install CHGNet yet)
    !pip install chgnet

In [10]:
!pip install chgnet

In [4]:
import numpy as np
from pymatgen.core import Structure

from chgnet.model import CHGNet

# If the above line fails in Google Colab due to numpy version issue,
# please restart the runtime, and the problem will be solved

np.set_printoptions(precision=4, suppress=True)

Read structure from a json or cif file

In [5]:
try:
    from chgnet import ROOT

    structure = Structure.from_file(f"{ROOT}/examples/mp-18767-LiMnO2.cif")
except Exception:
    from urllib.request import urlopen

    url = "https://raw.githubusercontent.com/CederGroupHub/chgnet/main/examples/mp-18767-LiMnO2.cif"
    cif = urlopen(url).read().decode("utf-8")
    structure = Structure.from_str(cif, fmt="cif")

print(structure)

Full Formula (Li2 Mn2 O4)
Reduced Formula: LiMnO2
abc   :   2.868779   4.634475   5.832507
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (8)
  #  SP      a    b         c
---  ----  ---  ---  --------
  0  Li+   0.5  0.5  0.37975
  1  Li+   0    0    0.62025
  2  Mn3+  0.5  0.5  0.863252
  3  Mn3+  0    0    0.136747
  4  O2-   0.5  0    0.360824
  5  O2-   0    0.5  0.098514
  6  O2-   0.5  0    0.901486
  7  O2-   0    0.5  0.639176


Define Model

In [14]:
chgnet = CHGNet.load()

# Alternatively you can read your own model
# chgnet = CHGNet.from_file(model_path)

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on mps


Predict energy, force, stress, magmom

In [16]:
prediction = chgnet.predict_structure(structure)

for key, unit in [
    ("energy", "eV/atom"),
    ("forces", "eV/A"),
    ("stress", "GPa"),
    ("magmom", "mu_B"),
]:
    print(f"CHGNet-predicted {key} ({unit}):\n{prediction[key[0]]}\n")

CHGNet-predicted energy (eV/atom):
-7.3676910400390625

CHGNet-predicted forces (eV/A):
[[-0.     -0.      0.0238]
 [-0.      0.     -0.0238]
 [ 0.      0.      0.0926]
 [-0.     -0.     -0.0926]
 [ 0.      0.     -0.0024]
 [ 0.      0.     -0.0131]
 [-0.     -0.      0.0131]
 [ 0.      0.      0.0024]]

CHGNet-predicted stress (GPa):
[[-0.304  -0.     -0.    ]
 [-0.      0.2231  0.    ]
 [-0.      0.     -0.1073]]

CHGNet-predicted magmom (mu_B):
[0.003  0.003  3.8694 3.8694 0.0441 0.0386 0.0386 0.0441]



Structure Optimization

In [17]:
from chgnet.model import StructOptimizer

relaxer = StructOptimizer()

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on mps


In [18]:
# Perturb the structure
structure.perturb(0.1)

# Relax the perturbed structure
result = relaxer.relax(structure, verbose=True)
print("Relaxed structure:\n")
print(result["final_structure"])

      Step     Time          Energy          fmax
FIRE:    0 16:41:41      -58.719028        0.241698
FIRE:    1 16:41:42      -58.719997        1.103130
FIRE:    2 16:41:42      -58.766087        0.902190
FIRE:    3 16:41:43      -58.829632        0.646016
FIRE:    4 16:41:43      -58.880054        0.448234
FIRE:    5 16:41:44      -58.902187        0.332600
FIRE:    6 16:41:44      -58.903545        0.451014
FIRE:    7 16:41:44      -58.905586        0.432571
FIRE:    8 16:41:44      -58.909386        0.395806
FIRE:    9 16:41:44      -58.914410        0.340308
FIRE:   10 16:41:45      -58.919991        0.290973
FIRE:   11 16:41:45      -58.925446        0.242369
FIRE:   12 16:41:45      -58.930099        0.197698
FIRE:   13 16:41:45      -58.933319        0.158006
FIRE:   14 16:41:46      -58.934971        0.135387
FIRE:   15 16:41:46      -58.935104        0.187229
FIRE:   16 16:41:46      -58.935200        0.184778
FIRE:   17 16:41:46      -58.935394        0.179963
FIRE:   18 16:

Molecular Dynamics

In [19]:
from chgnet.model.dynamics import MolecularDynamics

md = MolecularDynamics(
    atoms=structure,
    model=chgnet,
    ensemble="nvt",
    temperature=1000,  # in k
    timestep=2,  # in fs
    trajectory="md_out.traj",
    logfile="md_out.log",
    loginterval=100,
)
md.run(50)  # run a 0.1 ps MD simulation

CHGNet will run on mps
NVT-Berendsen-MD created


/opt/anaconda3/lib/python3.11/site-packages/ase/md/nvtberendsen.py:109: RuntimeWarning: divide by zero encountered in scalar divide
  (self.temperature / old_temperature - 1.0) *


Magmom Visualization

In [20]:
supercell = structure.make_supercell([2, 2, 2], in_place=False)
print(supercell.composition)

Li+16 Mn3+16 O2-32


In [21]:
import random

n_Li = int(supercell.composition["Li+"])
remove_ids = random.sample(list(range(n_Li)), n_Li // 2)

supercell.remove_sites(remove_ids)
print(supercell.composition)

Li+8 Mn3+16 O2-32


In [22]:
result = relaxer.relax(supercell)

      Step     Time          Energy          fmax
FIRE:    0 16:49:03     -424.481064        2.384429
FIRE:    1 16:49:06     -425.271496        1.764571
FIRE:    2 16:49:09     -426.254803        1.135107
FIRE:    3 16:49:10     -426.813480        1.054440
FIRE:    4 16:49:12     -426.965366        1.832455
FIRE:    5 16:49:13     -427.031723        1.665947
FIRE:    6 16:49:14     -427.144543        1.353258
FIRE:    7 16:49:15     -427.273624        0.942368
FIRE:    8 16:49:17     -427.391731        0.598631
FIRE:    9 16:49:19     -427.479904        0.525716
FIRE:   10 16:49:21     -427.538624        0.503314
FIRE:   11 16:49:23     -427.582870        0.677975
FIRE:   12 16:49:28     -427.638412        0.773737
FIRE:   13 16:49:30     -427.723381        0.810846
FIRE:   14 16:49:32     -427.842716        0.690752
FIRE:   15 16:49:42     -427.980370        0.462095
FIRE:   16 16:49:46     -428.100132        0.459664
FIRE:   17 16:49:49     -428.178585        0.496651
FIRE:   18 16:

KeyboardInterrupt: 